In [33]:
import pandas as pd
import lightgbm as lgb
import os
import gc
import pickle
from sklearn import preprocessing, model_selection

In [122]:
#"""
def get_type_feature_all_(sample, train_df, key, on, type_c, mark):
    if type_c == "count":
        tmp = pd.DataFrame(train_df[key + [on]].groupby(key)[on].count()).reset_index()
    if type_c == "mean":
        tmp = pd.DataFrame(train_df[key + [on]].groupby(key)[on].mean()).reset_index()
    if type_c == "nunique":
        tmp = pd.DataFrame(train_df[key + [on]].groupby(key)[on].nunique()).reset_index()
    if type_c == "max":
        tmp = pd.DataFrame(train_df[key + [on]].groupby(key)[on].max()).reset_index()
    if type_c == "min":
        tmp = pd.DataFrame(train_df[key + [on]].groupby(key)[on].min()).reset_index()
    if type_c == "sum":
        tmp = pd.DataFrame(train_df[key + [on]].groupby(key)[on].sum()).reset_index()
    if type_c == "std":
        tmp = pd.DataFrame(train_df[key + [on]].groupby(key)[on].std()).reset_index()
    if type_c == "median":
        tmp = pd.DataFrame(train_df[key + [on]].groupby(key)[on].median()).reset_index()
    tmp.columns = key + [mark + "_" + "_".join(key) + '_%s_' % type_c + on]
    tmp[mark + "_" + "_".join(key) + '_%s_' % type_c + on] = tmp[
        mark + "_" + "_".join(key) + '_%s_' % type_c + on].astype('float32')
    sample = sample.merge(tmp, on=key, how='left')
    del tmp, train_df
    gc.collect()
    return sample, mark + "_" + "_".join(key) + '_%s_' % type_c + on
#"""
def get_type_feature_all(sample,train_df,key,on,type_c,mark):
    pickle_path="../pickle/"
    filename = "_".join([mark+"_%s_features"%type_c, "_".join(key), on, str(len(sample))]) + ".pkl"
    try:
        with open(pickle_path + filename, "rb") as fp:
            print("load {} {} feature from pickle file: key: {}, on: {}...".format(mark,type_c,"_".join(key), on))
            col = pickle.load(fp)
        for c in col.columns:
            sample[c] = col[c]
        gc.collect()
    except:
        print('get {} {} feature, key: {}, on: {}'.format(mark,type_c,"_".join(key), on))
        if type_c=="count":
            tmp = pd.DataFrame(train_df[key+[on]].groupby(key)[on].count()).reset_index()
        if type_c=="mean":
            tmp = pd.DataFrame(train_df[key+[on]].groupby(key)[on].mean()).reset_index()
        if type_c=="nunique":
            tmp = pd.DataFrame(train_df[key+[on]].groupby(key)[on].nunique()).reset_index()
        if type_c=="max":
            tmp = pd.DataFrame(train_df[key+[on]].groupby(key)[on].max()).reset_index()
        if type_c=="min":
            tmp = pd.DataFrame(train_df[key+[on]].groupby(key)[on].min()).reset_index()
        if type_c=="sum":
            tmp = pd.DataFrame(train_df[key+[on]].groupby(key)[on].sum()).reset_index()
        if type_c=="std":
            tmp = pd.DataFrame(train_df[key+[on]].groupby(key)[on].std()).reset_index()
        if type_c=="median":
            tmp = pd.DataFrame(train_df[key+[on]].groupby(key)[on].median()).reset_index()
        tmp.columns = key+[mark+"_"+"_".join(key) + '_%s_'%type_c + on]
        tmp[mark+"_"+"_".join(key) + '_%s_'%type_c + on] = tmp[mark+"_"+"_".join(key) + '_%s_'%type_c + on].astype('float32')
        sample = sample.merge(tmp, on=key, how='left')
        with open(pickle_path + filename, "wb") as fp:
            col = sample[[mark+"_"+"_".join(key) + '_%s_'%type_c + on]].copy()
            pickle.dump(col, fp)
        del tmp
    #if save_only:
        #for i in col.columns:
            #del sample[i]
    del col,train_df
    gc.collect()
    return sample, mark + "_" + "_".join(key) + '_%s_' % type_c + on


In [123]:
train_data=pd.read_csv("../data/train.csv")
test_data=pd.read_csv("../data/test.csv")
sub=pd.read_csv("../data/sample_submission.csv")

In [124]:
def create_features(data):
    #data["order_1h_2h"]=data["orders_1h"]+data["orders_2h"]
    
    date_max=data.groupby(["date"]).article_id.max().reset_index()
    date_max.columns=["date","date_max"]
    data_min=date_max.copy()
    data_min.columns=["date","date_min"]
    data_min["date"]=data_min["date"]+1
    
    data=data.merge(date_max,on="date",how="left")
    data=data.merge(data_min,on="date",how="left")
    data["date_min"]=data["date_min"].fillna(0)
    
    data["article_id_date"]=(data["article_id"]-data["date_min"])/(data["date_max"]-data["date_min"])
    del data["date_min"],data["date_max"]
    return data

In [125]:
data=pd.concat([train_data,test_data]).reset_index(drop=True)
#data=create_features(data)

In [126]:
#train=train_data[train_data.date>=110].copy()#[['article_id', 'date','orders_1h','orders_2h','orders_3h_15h']].copy()
#test=test_data.copy()#[['article_id', 'date','orders_1h','orders_2h']].copy()

In [127]:
train=data[(data.date>=110)&(data.date<=116)].copy().reset_index(drop=True)
test=data[data.date>116].copy().reset_index(drop=True)

In [128]:
#train.article_id_date.mean(),test.article_id_date.mean()

In [129]:
#train=create_features(train)
#test=create_features(test)

In [130]:
train_stacking_ori=pd.read_csv("../user_data/train_stacking_1.0638693813136202.csv")
test_stacking_ori=pd.read_csv("../user_data/test_stacking_1.0638693813136202.csv")

train_stacking_reg=pd.read_csv("../user_data/train_stacking_4.269348471761833.csv")
train_stacking_reg.columns=['article_id', 'pred2']
test_stacking_reg=pd.read_csv("../user_data/test_stacking_4.269348471761833.csv")
test_stacking_reg.columns=['article_id', 'pred2']

train_stacking_multiclass=pd.read_csv("../user_data/train_stacking_multiclass.csv")
test_stacking_multiclass=pd.read_csv("../user_data/test_stacking_multiclass.csv")

train_stacking_nn_reg=pd.read_csv("../user_data/train_stacking_nn_reg.csv")
test_stacking_nn_reg=pd.read_csv("../user_data/test_stacking_nn_reg.csv")#

train_stacking_nn_multiclass=pd.read_csv("../user_data/train_stacking_nn_multiclass.csv")
test_stacking_nn_multiclass=pd.read_csv("../user_data/test_stacking_nn_multiclass.csv")#

train_stacking_multiclassova=pd.read_csv("../user_data/train_stacking_multiclassova.csv")
test_stacking_multiclassova=pd.read_csv("../user_data/test_stacking_multiclassova.csv")

train_ja=pd.read_csv("../user_data/ja_scion/ja_train_scion.csv")
test_ja=pd.read_csv("../user_data/ja_scion/ja_test_scion.csv")

train_stacking_model2=pd.read_csv("../user_data/train_stacking_1.062315379742038.csv")
train_stacking_model2.columns=['article_id', 'pred3']
test_stacking_model2=pd.read_csv("../user_data/test_stacking_1.062315379742038.csv")
test_stacking_model2.columns=['article_id', 'pred3']

train_stacking_nn_reg_2=pd.read_csv("../user_data/train_stacking_nn_reg_2.csv")
test_stacking_nn_reg_2=pd.read_csv("../user_data/test_stacking_nn_reg_2.csv")#

In [131]:

train=train.merge(train_stacking_ori,on="article_id",how="left")
test=test.merge(test_stacking_ori,on="article_id",how="left")

train=train.merge(train_stacking_reg,on="article_id",how="left")
test=test.merge(test_stacking_reg,on="article_id",how="left")

train=train.merge(train_stacking_multiclass,on="article_id",how="left")
test=test.merge(test_stacking_multiclass,on="article_id",how="left")

train=train.merge(train_stacking_nn_reg,on="article_id",how="left")
test=test.merge(test_stacking_nn_reg,on="article_id",how="left")

train=train.merge(train_stacking_nn_multiclass,on="article_id",how="left")
test=test.merge(test_stacking_nn_multiclass,on="article_id",how="left")

train=train.merge(train_stacking_multiclassova,on="article_id",how="left")
test=test.merge(test_stacking_multiclassova,on="article_id",how="left")

train=train.merge(train_stacking_model2,on="article_id",how="left")
test=test.merge(test_stacking_model2,on="article_id",how="left")

train=train.merge(train_stacking_nn_reg_2,on="article_id",how="left")
test=test.merge(test_stacking_nn_reg_2,on="article_id",how="left")

train=train.merge(train_ja,on="article_id",how="left")
test=test.merge(test_ja,on="article_id",how="left")

In [132]:
score=train[train.date>=115].copy()
print(((score["pred"]-score["orders_3h_15h"])**2).mean())

1.1023229683034188


In [133]:
#1.1023229683034188

In [134]:
train.head()

,article_id,date,baike_id_1h,price,price_diff,author,level1,level2,level3,level4,...,ja3,ja4,ja5,ja6,ja7,ja8,ja9,ja10,ja11,ja12
0,1664040,110,175435,284.00,NaN,48906,9,70,414,29,...,0.039223,0.053054,0.081302,0.119573,0.307969,0.305706,0.065088,2.208236e-04,5.029412e-174,0.0
1,1664289,110,219783,23.80,NaN,39437,19,2,752,0,...,0.870572,0.113277,0.037071,0.007063,0.004393,0.000739,0.000015,1.346449e-08,0.000000e+00,0.0
2,1664686,110,32815,19.90,7.0,52372,3,82,55,1004,...,0.018873,0.023217,0.026392,0.031700,0.246764,0.604190,0.130894,8.592879e-03,3.848313e-48,0.0
3,1666137,110,219783,35.67,NaN,59879,3,82,53,0,...,0.826350,0.125068,0.058818,0.009125,0.007058,0.000490,0.000043,9.965661e-07,0.000000e+00,0.0
4,1667990,110,30928,1999.00,0.0,62460,9,67,359,28,...,0.578761,0.223882,0.120942,0.051146,0.041429,0.000525,0.000155,2.395418e-08,0.000000e+00,0.0


In [135]:
train_cv=train[(train.date<115)].copy()
valid_cv=train[(train.date>=115)].copy()

In [136]:
features = list(train[:1].drop(['date','orders_3h_15h'],axis=1).columns)

In [137]:
train_cv.shape,valid_cv.shape

((105434, 76), (39527, 76))

In [142]:
train_set = lgb.Dataset(train_cv[features], label=train_cv["orders_3h_15h"])
val_set = lgb.Dataset(valid_cv[features], label=valid_cv["orders_3h_15h"])

params_lgbm = {
          'task': 'train',
          'boosting_type': 'gbdt',
          'objective': 'regression',
          'metric': 'mse',
          'min_child_weight': 1.5,
          'num_leaves': 2**5,
          'max_depth': 5,
          'lambda_l2': 0.7,
          'subsample': 0.7,
          'colsample_bytree': 0.7,
          'colsample_bylevel': 0.7,
          'learning_rate': 0.005,
          #'tree_method': 'exact',
          'seed': 2021,
          'nthread': 18,
          'verbose': -1,
          }


model = lgb.train(params_lgbm, 
                  train_set, 
                  1000, 
                  valid_sets=val_set, 
                  #feval=feval_RMSPE,
                  verbose_eval= 500,
                  #early_stopping_rounds=500,
                  #categorical_feature=cats,
                 )

[LightGBM] [Warning] Unknown parameter: colsample_bylevel
[500]	valid_0's l2: 1.06022
[1000]	valid_0's l2: 1.03457


In [143]:
#1.0912236723940199
#[1000]	valid_0's l2: 1.09141
#[1000]	valid_0's l2: 1.08866    +nn
#[1000]	valid_0's l2: 1.08638    +nn  multiclass
#[1000]	valid_0's l2: 1.08467    +lgb multiclass ova
#[1000]	valid_0's l2: 1.07189
#[1000]	valid_0's l2: 1.06989

In [144]:
valid_score=valid_cv[['article_id', 'date','orders_3h_15h']].copy()
valid_score["pred"]=model.predict(valid_cv[features])
valid_score["pred"]=valid_score["pred"].apply(lambda x:0 if x<0 else x)
score=((valid_score["pred"]-valid_score["orders_3h_15h"])**2).mean()
print(score)

1.0345648716842124


In [113]:
for i in range(20):
    valid_score_a=valid_score.sample(frac=0.3)
    valid_score_b=valid_score[~valid_score.index.isin(valid_score_a.index)]
    score_a=((valid_score_a["pred"]-valid_score_a["orders_3h_15h"])**2).mean()
    score_b=((valid_score_b["pred"]-valid_score_b["orders_3h_15h"])**2).mean()
    mean_a=valid_score_a["orders_3h_15h"].mean()
    mean_b=valid_score_b["orders_3h_15h"].mean()
    print(score_a,score_b,mean_a,mean_b)

1.0805581896677496 1.0612644683543724 0.9689660988362288 0.9418844193863168
1.0784580651220836 1.062164510887983 0.9484736043177602 0.9506668112327876
1.0681953653574296 1.0665627578000223 0.9564850733681902 0.9472333658607105
1.0515859301249328 1.0736810014296019 0.9402934727610053 0.9541725396653294
1.0815370260750123 1.0608449720907762 0.9456906729634003 0.9518594817304564
1.0594569341178184 1.0703077546426745 0.9413054477989543 0.9537388413025407
1.0785231434921854 1.0621366204939415 0.9440040479001518 0.9525823123351043
1.0504874758878302 1.0741517618598198 0.9567380671276775 0.9471249412700133
1.0625353120448429 1.0689884657107096 0.9408837915331422 0.9539195489537027
1.0847089399397367 1.0594855968113 0.9653398549502445 0.9434385051863096
1.082091328079861 1.0606074169506001 0.9646652049249452 0.9437276374281687
1.1128207199456779 1.0474378362377132 0.9595209984820374 0.9459322707723445
1.0679229976357523 1.0666794854173458 0.938438185191432 0.954967653330442
1.0483417605346559 

1.0788363317159122 1.0620023985865052 0.9373418789003205 0.955437493223463


In [146]:
score

1.0345648716842124

In [145]:
print("\n".join(("%s: %.2f" % x) for x in
                list(sorted(zip(features, model.feature_importance("gain")), key=lambda x: x[1],
                      
                            reverse=True))[:200]
                ))


ja1: 11659514.61
ja3: 1715318.94
pred3: 1285448.70
pred: 1224250.50
ja2: 393693.78
pred_nn_4: 83633.90
multiclassova_0: 69777.11
article_id: 58814.66
pred_nn: 57604.95
level2: 37368.14
multiclass_0: 32198.09
multiclass_2: 27159.34
ja9: 20234.93
level1: 19723.08
price_diff: 18534.37
pred_nn_3: 18454.84
multiclass_5: 16636.49
multiclass_1: 16593.35
multiclass_4: 15144.53
pred2: 15095.22
price: 14510.87
multiclassova_7: 14404.02
mall: 14289.51
pred_nn_0: 13495.03
baike_id_1h: 13380.70
ja4: 12997.39
pred_nn_2: 12469.92
url: 12056.23
ja10: 10761.38
nn_multiclass_5: 10441.02
level4: 10408.35
multiclass_7: 10229.41
ja8: 9428.95
nn_multiclass_3: 9400.83
multiclassova_2: 9282.08
multiclass_6: 9102.71
multiclassova_5: 8832.26
pred_nn_1: 8696.95
nn_multiclass_6: 8573.26
nn_multiclass_1: 8199.61
level3: 7550.25
multiclass_3: 7488.33
ja6: 7388.86
multiclass_8: 7246.03
brand: 7198.86
nn_multiclass_4: 7071.42
multiclassova_1: 6580.01
author: 6547.79
multiclassova_4: 6244.29
multiclassova_3: 6154.16
n

In [147]:
train_set = lgb.Dataset(train[features], label=train["orders_3h_15h"])
val_set = lgb.Dataset(valid_cv[features], label=valid_cv["orders_3h_15h"])

params_lgbm = {
          'task': 'train',
          'boosting_type': 'gbdt',
          'objective': 'regression',
          'metric': 'mse',
          'min_child_weight': 1.5,
          'num_leaves': 2**5,
          'max_depth': 5,
          'lambda_l2': 0.7,
          'subsample': 0.7,
          'colsample_bytree': 0.7,
          'colsample_bylevel': 0.7,
          'learning_rate': 0.005,
          #'tree_method': 'exact',
          'seed': 2021,
          'nthread': 18,
          'verbose': -1,
          }


model = lgb.train(params_lgbm, 
                  train_set, 
                  1000, 
                  valid_sets=val_set, 
                  #feval=feval_RMSPE,
                  verbose_eval= 500,
                  #early_stopping_rounds=500,
                  #categorical_feature=cats,
                 )

[LightGBM] [Warning] Unknown parameter: colsample_bylevel
[500]	valid_0's l2: 1.03678
[1000]	valid_0's l2: 0.995379


In [148]:
submission=test[["article_id"]].copy()
submission["orders_3h_15h"]=model.predict(test[features])
submission["orders_3h_15h"]=submission["orders_3h_15h"].apply(lambda x:0 if x<0 else x)

submission.to_csv("../sub/summission_%s.csv"%score,index=None)
submission.to_csv("../sub/summission.csv",index=None)

In [149]:
submission.shape

(149589, 2)

In [150]:
submission.orders_3h_15h.mean()

0.7748418956511023

In [151]:
test.shape

(149589, 76)

In [152]:
test.pred.mean()

0.8046991951966016

In [153]:
train.pred.mean()

0.8784951391982075